In [1]:
import pandas as pd

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
tonight_odds

,date,visitor,local,v_line,l_line
0,2024-02-04,ORL,DET,1.387597,3.1
1,2024-02-04,PHO,WAS,1.166667,5.4


In [4]:
prev_odds

,date,visitor,local,v_line,l_line
0,2024-02-03,BKN,PHI,2.420000,1.588235
1,2024-02-03,GSW,ATL,2.200000,1.704225
2,2024-02-03,SAC,CHI,2.050000,1.800000
3,2024-02-03,LAL,NYK,2.640000,1.505051
4,2024-02-03,MIL,DAL,1.540541,2.540000
5,2024-02-03,CLE,SAS,1.219780,4.500000


In [5]:
prev_odds['visitor'] = prev_odds['visitor'].apply(lambda x: x.replace('BKN', 'BRK'))

In [6]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [7]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [8]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,50,2024-02-04,2,111.571429,97.589796,0.530592,12.271429,25.975510,0.234000,113.614286,...,13.156250,25.106250,0.197437,112.595833,0,2,6,2,22,2024
1,50,2024-02-04,-1,116.877551,97.246939,0.567878,12.616327,25.004082,0.239592,119.616327,...,11.658333,19.639583,0.182208,112.156250,0,2,6,2,24,2024


In [9]:
import joblib

In [11]:
scaler = joblib.load('Models/Scalers/LR.bin')
scaled_df = scaler.transform(games_df.drop(columns=['Date']))

In [13]:
model = joblib.load('Models/TrainedModels/LR_Model.pkl')
pred = model.predict(scaled_df)

In [15]:
tonight_odds['Predictions'] = pred
tonight_odds

,date,visitor,local,v_line,l_line,Predictions
0,2024-02-04,ORL,DET,1.387597,3.1,1
1,2024-02-04,PHO,WAS,1.166667,5.4,1


In [ ]:
tonight_odds.to_excel('predictions.xlsx', index=0)